## <span style="color:#0b486b">Naive Bayes Classifier</span> 


Naive Bayes is one of the most practical classification machine learning algorithms. 

* fast
* good performance
* simple yet very effective
* robust to irrelative features

So why is it called naive?

Because it does not consider the dependency between features and assume all features are independent of each other which is not the case in reality. This is a naive assumption, hence the name.

The accuracy is very good although this naive assumption. A famous example of NB usage is spam filtering.

---
### Example1

We assume we have collected the below data for the past 5 days. Based on this data, can we predict if our subject will play in a setting like:

    outlook  = overcast
    temp     = hot
    humidity = normal
    windy    = no
    
<table border=1>
  <tr>
    <th>Outlook</th>
    <th>Temperature</th>
    <th>Humidity</th>
    <th>Windy</th>
    <th>Play</th>
  </tr>
  <tr>
    <td>sunny</td>
    <td>hot</td>
    <td>normal</td>
    <td>no</td>
    <td>yes</td>
  </tr>
  <tr>
    <td>overcast</td>
    <td>mild</td>
    <td>normal</td>
    <td>no</td>
    <td>yes</td>
  </tr>
    <tr>
    <td>rainy</td>
    <td>cool</td>
    <td>high</td>
    <td>yes</td>
    <td>no</td>
  </tr>
    <tr>
    <td>sunny</td>
    <td>mild</td>
    <td>normal</td>
    <td>yes</td>
    <td>no</td>
  </tr>
    <tr>
    <td>overcast</td>
    <td>hot</td>
    <td>high</td>
    <td>no</td>
    <td>no</td>
  </tr>
</table>
    


First we have to find a representation for our data. We can construct a dictionary to convert stings into numbers and then save them in a dataframe. 

    outlook: sunny=0, overcast=1, rainy=2
    temp: hot=0, mild=1, cool=2
    humidity: normal=0, high=1
    wind: no=0, yes=1
    play: np=0, yes=1

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = {
    'outlook': [0, 1, 2, 0, 1],
    'temp'   : [0, 1, 2, 1, 0],
    'humid'  : [0, 0, 1, 0, 1],
    'wind'   : [0, 0, 1, 1, 0],
    'play'   : [1, 1, 0, 0, 0,]    
}

df = pd.DataFrame(data)

Now we use Bayes rule to construct a Naive Bayes classifier. We can write:

$$Pr\left(p|o,t,h,w\right)\propto Pr\left(p\right)Pr(o|p)Pr(t|p)Pr(h|p)Pr(w|p)$$

To calculate $Pr(p)$ we use marginal probablity.

In [3]:
def marginal_prob(df, column):
    '''
    Compute the marignal probability for values in a column
    '''
    # an array contain pairs of (value, count)
    vals_counts = [(val, (df[column] == val).sum()) for val in set(df[column])]
    print(vals_counts)
    total_count = sum([count for val, count in vals_counts])
    
    # an array contain pairs of (value, probability)
    vals_probs = [(val, count/total_count) for val, count in vals_counts]
    # a dictionary in which keys are val and values are the corresponding probabilities
    return dict(vals_probs)

print(marginal_prob(df,'outlook'))

print(set(df['outlook']))



[(0, 2), (1, 2), (2, 1)]
{0: 0.4, 1: 0.4, 2: 0.2}
{0, 1, 2}


To calculate probability of a feature given the class (play) we use conditinoal probability.

In [4]:
def conditional_prob(df, feature, c, val):
    # c is the class (0 or 1)
    df2 = df[df[c] == val][feature]
    vals_counts = [[val, (df2 == val).sum() + 1e-8] for val in set(df[feature])]
    total_count = sum([count for val, count in vals_counts])
    
    vals_probs = [(val, count/total_count) for val, count in vals_counts]
    return dict(vals_probs)


conditional_prob(df,'outlook','play',1)

{0: 0.49999999750000007, 1: 0.49999999750000007, 2: 4.999999925000001e-09}

Now we can use Bayes rule:

In [5]:
o = 1
t = 0
h = 0
w = 0

c = 0
p0 = marginal_prob(df, 'play')[c] * conditional_prob(df, 'outlook', 'play', c)[o] * conditional_prob(df, 'temp', 'play', c)[t] \
* conditional_prob(df, 'humid', 'play', c)[h] * conditional_prob(df, 'wind', 'play', c)[w]

c = 1
p1 = marginal_prob(df, 'play')[c] * conditional_prob(df, 'outlook', 'play', c)[o] * conditional_prob(df, 'temp', 'play', c)[t] \
* conditional_prob(df, 'humid', 'play', c)[h] * conditional_prob(df, 'wind', 'play', c)[w]

# normalizing
p_sum = p0 + p1
p0 /= p_sum
p1 /= p_sum

print("probability of not playing: {}".format(p0))
print("probability of playing    : {}".format(p1))

[(0, 3), (1, 2)]
[(0, 3), (1, 2)]
probability of not playing: 0.0689655189536266
probability of playing    : 0.9310344810463733


---
### Example 2

Suppose we have documents below as our training set. 

    d1: Chinese Beijing Chinese , class = C
    d2: Chinese Chinese Shanghai, class = C
    d3: Chinese Macao           , class = C
    d4: Tokyo Japan Chinese     , class = J


**Exercise:** Train a NB classifier and predict if `d5` belongs to class C or J.

    d5: Chinese Chinese Tokyo Japan, class = ?
    
What about `d6: Chinese Chinese Chinese Tokyo Japan, class = ?`

In [6]:
init_train_data = {"d1": [["Chinese","Beijing","Chinese"], "C"],
"d2": [["Chinese","Chinese","Shanghai"], "C"],
"d3": [["Chinese","Macao"], "C"],
"d4": [["Tokyo","Japan","Chinese"], "J"]
}


In [7]:
word_histogram = {}
class_count = {}

for key, val in init_train_data.items():
    words = val[0]
    class_val = val[1]
    if class_val in word_histogram:
        for i in words:
            if i in word_histogram[class_val]:
                word_histogram[class_val][i] += 1
            else:
                word_histogram[class_val][i] = 1
        class_count[class_val] += 1
    else:
        word_histogram[class_val] = {}
        for i in words:
            if i in word_histogram[class_val]:
                word_histogram[class_val][i] += 1
            else:
                word_histogram[class_val][i] = 1
        class_count[class_val] = 1


## If some words are not include all the classes, add missing words to those classes with count 1 and increase the other word count 
word_set = []

for i in init_train_data.values():
    word_set += i[0]

word_set = list(set(word_set))



for clss in word_histogram.keys():
    not_include_words = []
    for word in word_set:
        if word not in word_histogram[clss]:
            not_include_words.append(word)
    for key, val in word_histogram[clss].items():
        # 1)  In here, I am adding count of new words to each words that are already available in the class. In this case, both d5 and d6 belongs to class J
        #     if need to change, replace as :  word_histogram[clss][key] = val + 1 
        # 2)  But instead of adding count of new words, if I add just 1 to already available words in the class, 
        #     probabilities of C and J change and both d5 and d6 belongs to class C
        word_histogram[clss][key] = val + len(not_include_words) 
    for i in not_include_words:
        word_histogram[clss][i] = 1

print(word_histogram)
print(class_count)
   

{'C': {'Chinese': 7, 'Beijing': 3, 'Shanghai': 3, 'Macao': 3, 'Japan': 1, 'Tokyo': 1}, 'J': {'Tokyo': 4, 'Japan': 4, 'Chinese': 4, 'Shanghai': 1, 'Beijing': 1, 'Macao': 1}}
{'C': 3, 'J': 1}


In [8]:
def class_prob(clss, clss_cnt = class_count):
    return clss_cnt[clss]/sum([i for i in clss_cnt.values()])

def class_word_prob(word, clss, clss_word_cnt = word_histogram):
    return clss_word_cnt[clss][word]/sum([i for i in clss_word_cnt[clss].values()])

#### d5: Chinese Chinese Tokyo Japan, class = ?

$$P_C \propto Pr(C)*Pr(Chinese|C)^2*Pr(Tokyo|C)*Pr(Japan|C) $$
$$P_J \propto Pr(J)*Pr(Chinese|J)^2*Pr(Tokyo|J)*Pr(Japan|J) $$

In [9]:
P_C = class_prob("C")*(class_word_prob("Chinese","C")**2)*class_word_prob("Tokyo","C")*class_word_prob("Japan","C")
P_J = class_prob("J")*(class_word_prob("Chinese","J")**2)*class_word_prob("Tokyo","J")*class_word_prob("Japan","J")

# Normalize the values

p_sum = P_C + P_J

P_C = P_C/p_sum
P_J = P_J/p_sum

print("probability of being class C: {}".format(P_C))
print("probability of being class J: {}".format(P_J))


probability of being class C: 0.21686482505647336
probability of being class J: 0.7831351749435267


#### d6: Chinese Chinese Chinese Tokyo Japan, class = ?

$$P_C \propto Pr(C)*Pr(Chinese|C)^3*Pr(Tokyo|C)*Pr(Japan|C) $$
$$P_J \propto Pr(J)*Pr(Chinese|J)^3*Pr(Tokyo|J)*Pr(Japan|J) $$

In [10]:
P_C = class_prob("C")*(class_word_prob("Chinese","C")**3)*class_word_prob("Tokyo","C")*class_word_prob("Japan","C")
P_J = class_prob("J")*(class_word_prob("Chinese","J")**3)*class_word_prob("Tokyo","J")*class_word_prob("Japan","J")

# Normalize the values

p_sum = P_C + P_J

P_C = P_C/p_sum
P_J = P_J/p_sum

print("probability of being class C: {}".format(P_C))
print("probability of being class J: {}".format(P_J))

probability of being class C: 0.28766804174786226
probability of being class J: 0.7123319582521377
